In [1]:
from __future__ import print_function,division
import utils; reload(utils)
from utils import *

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


###Header 3

In [2]:
import os
from glob import glob
from shutil import copyfile
import numpy as np

In [3]:
%matplotlib inline
path="/home/ubuntu/nbs/data/state/"
sample_path="/home/ubuntu/nbs/data/state/sample/"

In [4]:
batch_size=64

#### The validation set driver images should not be used in training Thus we have to group and copy over images for specific driver images to the validation folder using pandas group by

In [5]:
df = pd.read_csv(path + 'driver_imgs_list.csv')

#### Print out the number of drivers: expect 26

In [6]:
gp = df.groupby('subject')
len(gp.groups)

26

####  This function will move images for a fraction of drivers to the validation folder. 
#### It returns the total number of files moved

In [7]:
import pandas as pd
def create_validation_set(split):
    # read the CSV file into a pandas DataFrame
    df = pd.read_csv(path + 'driver_imgs_list.csv')
    # group the frame by the subject in the image
    gp = df.groupby('subject')
    # shuffle the list  of subjects
    
    shuffled_keys = np.random.permutation( gp.groups.keys())
    driver_count=len(gp.groups);
    validation_count = int(round(split*driver_count))
    filecount=0
    # pick a fraction of  drivers from the shuffled list
    for i in range(validation_count):
        subject=shuffled_keys[i]
        print('for driver {}'.format(subject))
        # get the group associated with the subject
        group = gp.get_group(subject)
        # loop over the group and move the images into the validation directory
        for (subject, cls, img) in group.values:
            source = '{}train/{}/{}'.format(path, cls, img)
            target = source.replace('train', 'valid')
            #print('mv {} {}'.format(source, target))
            filecount +=1
            os.rename(source, target)
    return filecount

In [ ]:
%mkdir data/state/valid

In [7]:
%cd  data/state/train

/home/ubuntu/nbs/data/state/train


In [ ]:
# for each c* subdirectory, in the main training folder
for d in glob('c?'):
    # create a sub dir in main valid folder
    os.mkdir('../valid/'+d)


In [ ]:
# this will move 20% of the drivers (their images) to the validation set
# assumption: each driver image set is of similar size so ~ 20% data moves to validation
#adjust the split  if observed count is less in validation
moved=create_validation_set(.2)
moved

In [ ]:
%cd  data/state/train
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

In [ ]:
# for each c* subdirectory, in the main training folder
for d in glob('c?'):
    # create a sub dir in sample/training folder
    os.mkdir('../sample/train/'+d)
    # create a sub dir in sample/valid folder
    os.mkdir('../sample/valid/'+d)

In [ ]:
# read the list of images   from each c sub folder in the main training directory into an array
imgs= glob('c?/*.jpg')
#Randomly shuffle this list
shuffled_imgs = np.random.permutation(imgs)
# pick first 1500 files from the shuffled list and copy to the sample/training/c* sub directories
for i in range(1500): copyfile(shuffled_imgs[i],'../sample/train/'+shuffled_imgs[i])


In [ ]:
# move to the main validation set  directory
%cd ../valid    

In [ ]:
# Read and randomly shuffle main validation folder c* sub directory images
imgs= glob('c?/*.jpg')
shuffled_imgs = np.random.permutation(imgs)
# copy first 1000 images over to sample/valid
for i in range(1000): copyfile(shuffled_imgs[i],'../sample/valid/'+shuffled_imgs[i])
shuffled_imgs[0]

In [ ]:
#create a results directory under the main state folder
%cd ../../..
%mkdir data/state/results

#create a test directory under sample folder
%mkdir data/state/sample/test

In [10]:
# get the training directoryIterator  from sample/train
batches=get_batches(sample_path+'train',batch_size=batch_size)
# get the validation  directory iterator from sample/valid .
# Batch size double of train(more memory available)
val_batches=get_batches(sample_path+'valid',batch_size=batch_size*2,shuffle=False)


Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [10]:
# linear model
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])

In [11]:
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches,
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 36s - loss: 13.0878 - acc: 0.1387 - val_loss: 13.0909 - val_acc: 0.1790
Epoch 2/2
1500/1500 [==============================] - 27s - loss: 13.1643 - acc: 0.1740 - val_loss: 12.8194 - val_acc: 0.2000


#### The model is predicting mostly 2-3 classes with 20% validation accuracy. But in past runs, validation accuracy was seen at 9%, with only 1 class being predicted all the time. 

In [12]:
np.round(model.predict_generator(batches, batches.N)[:10],2)

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.]], dtype=float32)

In [13]:
model.optimizer.lr=1e-10
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 32s - loss: 13.1248 - acc: 0.1807 - val_loss: 12.6892 - val_acc: 0.2120
Epoch 2/2
1500/1500 [==============================] - 24s - loss: 13.0386 - acc: 0.1887 - val_loss: 12.7757 - val_acc: 0.2030


#### Not much change- let us increase the learning rate

In [14]:
model.optimizer.lr=1e-5
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/4
1500/1500 [==============================] - 32s - loss: 13.1748 - acc: 0.1780 - val_loss: 12.8156 - val_acc: 0.2050
Epoch 2/4
1500/1500 [==============================] - 25s - loss: 13.0518 - acc: 0.1853 - val_loss: 12.6816 - val_acc: 0.2130
Epoch 3/4
1500/1500 [==============================] - 24s - loss: 13.2970 - acc: 0.1733 - val_loss: 12.6664 - val_acc: 0.2140
Epoch 4/4
1500/1500 [==============================] - 25s - loss: 12.8738 - acc: 0.2000 - val_loss: 12.8091 - val_acc: 0.2050


#### Clearly, with the drivers being completely different, the training error and validation set error are both high. We are underfitting and need a more complex model.  This is NOT what Jeremy had seen though- but my guess is he had not used different drivers not seen in training.  The new model gets us to 40% accuracy

In [15]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 32s - loss: 2.0536 - acc: 0.3313 - val_loss: 4.3982 - val_acc: 0.1330
Epoch 2/2
1500/1500 [==============================] - 24s - loss: 1.1208 - acc: 0.6893 - val_loss: 1.8862 - val_acc: 0.4000


#### now we seem to be  headed in the right direction, increase the learning rate

In [15]:
model.optimizer.lr = 0.01
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/5
1500/1500 [==============================] - 32s - loss: 0.7409 - acc: 0.8347 - val_loss: 1.6571 - val_acc: 0.4700
Epoch 2/5
1500/1500 [==============================] - 24s - loss: 0.4999 - acc: 0.9160 - val_loss: 1.1364 - val_acc: 0.5740
Epoch 3/5
1500/1500 [==============================] - 24s - loss: 0.3396 - acc: 0.9613 - val_loss: 0.9279 - val_acc: 0.7230
Epoch 4/5
1500/1500 [==============================] - 24s - loss: 0.2448 - acc: 0.9827 - val_loss: 0.8295 - val_acc: 0.7470
Epoch 5/5
1500/1500 [==============================] - 24s - loss: 0.1860 - acc: 0.9927 - val_loss: 0.7549 - val_acc: 0.8000


#### Moving to a conv model to see if accuracy can get better.

In [8]:
def with_conv1(batches):
    model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        #added conv layers
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D((3,3)),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D((3,3)),
        #end additions
        Flatten(),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])
    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)
    
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [13]:
with_conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 36s - loss: 1.6591 - acc: 0.5187 - val_loss: 2.2758 - val_acc: 0.3010
Epoch 2/2
1500/1500 [==============================] - 27s - loss: 0.3483 - acc: 0.9320 - val_loss: 2.0174 - val_acc: 0.2560
Epoch 1/4
1500/1500 [==============================] - 36s - loss: 0.0987 - acc: 0.9907 - val_loss: 2.4638 - val_acc: 0.2070
Epoch 2/4
1500/1500 [==============================] - 29s - loss: 0.0413 - acc: 0.9987 - val_loss: 2.7088 - val_acc: 0.2020
Epoch 3/4
1500/1500 [==============================] - 31s - loss: 0.0212 - acc: 1.0000 - val_loss: 2.8952 - val_acc: 0.1970
Epoch 4/4
1500/1500 [==============================] - 28s - loss: 0.0153 - acc: 1.0000 - val_loss: 2.9087 - val_acc: 0.1920


#### Augmentation  : shift

In [14]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1)
batches = get_batches(sample_path+'train', gen_t, batch_size=batch_size)

Found 1500 images belonging to 10 classes.


In [15]:
model = with_conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 38s - loss: 2.1894 - acc: 0.3267 - val_loss: 2.8344 - val_acc: 0.0930
Epoch 2/2
1500/1500 [==============================] - 30s - loss: 1.1870 - acc: 0.6407 - val_loss: 2.2089 - val_acc: 0.3760
Epoch 1/4
1500/1500 [==============================] - 36s - loss: 0.7911 - acc: 0.7653 - val_loss: 2.3160 - val_acc: 0.3840
Epoch 2/4
1500/1500 [==============================] - 30s - loss: 0.6105 - acc: 0.8287 - val_loss: 2.4959 - val_acc: 0.2420
Epoch 3/4
1500/1500 [==============================] - 27s - loss: 0.4640 - acc: 0.8760 - val_loss: 2.5909 - val_acc: 0.2620
Epoch 4/4
1500/1500 [==============================] - 32s - loss: 0.3705 - acc: 0.9113 - val_loss: 2.5639 - val_acc: 0.2840


#### Augmentation: height shift

In [16]:
gen_t = image.ImageDataGenerator(height_shift_range=0.05)
batches = get_batches(sample_path+'train', gen_t, batch_size=batch_size)

Found 1500 images belonging to 10 classes.


In [17]:
model = with_conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 35s - loss: 1.9241 - acc: 0.4060 - val_loss: 2.2981 - val_acc: 0.2170
Epoch 2/2
1500/1500 [==============================] - 31s - loss: 0.7790 - acc: 0.7827 - val_loss: 1.9247 - val_acc: 0.2760
Epoch 1/4
1500/1500 [==============================] - 37s - loss: 0.4189 - acc: 0.8860 - val_loss: 2.4460 - val_acc: 0.1350
Epoch 2/4
1500/1500 [==============================] - 28s - loss: 0.2960 - acc: 0.9247 - val_loss: 2.7683 - val_acc: 0.1380
Epoch 3/4
1500/1500 [==============================] - 28s - loss: 0.2098 - acc: 0.9513 - val_loss: 3.0525 - val_acc: 0.1640
Epoch 4/4
1500/1500 [==============================] - 30s - loss: 0.1478 - acc: 0.9707 - val_loss: 3.1981 - val_acc: 0.2200


#### This is taken as is from Jeremy's notebook-we apply all data augmentation together.

In [18]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(sample_path+'train', gen_t, batch_size=batch_size)

Found 1500 images belonging to 10 classes.


#### We also noticed so far that increasing the learning rate worsened the validation accuracy. We need the original smaller learning rate

In [19]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/5
1500/1500 [==============================] - 38s - loss: 1.7029 - acc: 0.4807 - val_loss: 3.0242 - val_acc: 0.2380
Epoch 2/5
1500/1500 [==============================] - 32s - loss: 1.3922 - acc: 0.5373 - val_loss: 2.8365 - val_acc: 0.2190
Epoch 3/5
1500/1500 [==============================] - 29s - loss: 1.2028 - acc: 0.6173 - val_loss: 2.4380 - val_acc: 0.2750
Epoch 4/5
1500/1500 [==============================] - 27s - loss: 1.0445 - acc: 0.6700 - val_loss: 2.0852 - val_acc: 0.3190
Epoch 5/5
1500/1500 [==============================] - 32s - loss: 1.0182 - acc: 0.6820 - val_loss: 1.8515 - val_acc: 0.3580


#### Use the same learning rate and keep going for more epochs

In [20]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=25, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/25
1500/1500 [==============================] - 37s - loss: 0.8896 - acc: 0.7007 - val_loss: 1.9342 - val_acc: 0.3490
Epoch 2/25
1500/1500 [==============================] - 30s - loss: 0.9068 - acc: 0.6980 - val_loss: 1.4757 - val_acc: 0.5210
Epoch 3/25
1500/1500 [==============================] - 30s - loss: 0.8038 - acc: 0.7373 - val_loss: 1.1117 - val_acc: 0.6290
Epoch 4/25
1500/1500 [==============================] - 30s - loss: 0.7483 - acc: 0.7693 - val_loss: 0.9926 - val_acc: 0.6860
Epoch 5/25
1500/1500 [==============================] - 30s - loss: 0.7198 - acc: 0.7747 - val_loss: 1.0209 - val_acc: 0.6710
Epoch 6/25
1500/1500 [==============================] - 29s - loss: 0.7312 - acc: 0.7827 - val_loss: 0.7500 - val_acc: 0.7890
Epoch 7/25
1500/1500 [==============================] - 32s - loss: 0.6303 - acc: 0.8127 - val_loss: 0.6576 - val_acc: 0.7910
Epoch 8/25
1500/1500 [==============================] - 34s - loss: 0.5909 - acc: 0.8200 - val_loss: 0.5140 - val_acc:

#### Now verify model on full dataset

In [9]:
# get the training directoryIterator  from sample/train
batches=get_batches(path+'train',batch_size=batch_size)
# get the validation  directory iterator from sample/valid .
# Batch size double of train(more memory available)
val_batches=get_batches(path+'valid',batch_size=batch_size*2,shuffle=False)


Found 18009 images belonging to 10 classes.
Found 4415 images belonging to 10 classes.


In [12]:
 model = with_conv1(batches)

Epoch 1/2
18009/18009 [==============================] - 306s - loss: 1.2782 - acc: 0.5869 - val_loss: 2.4741 - val_acc: 0.2600
Epoch 2/2
18009/18009 [==============================] - 292s - loss: 0.6603 - acc: 0.8009 - val_loss: 1.7717 - val_acc: 0.4632
Epoch 1/4
18009/18009 [==============================] - 297s - loss: 0.4545 - acc: 0.8680 - val_loss: 1.4119 - val_acc: 0.6263
Epoch 2/4
18009/18009 [==============================] - 290s - loss: 0.3387 - acc: 0.9067 - val_loss: 1.4742 - val_acc: 0.5767
Epoch 3/4
18009/18009 [==============================] - 293s - loss: 0.2746 - acc: 0.9259 - val_loss: 1.5096 - val_acc: 0.5622
Epoch 4/4
18009/18009 [==============================] - 295s - loss: 0.2268 - acc: 0.9403 - val_loss: 1.3859 - val_acc: 0.6344


#### Data augmentation as before

In [13]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 18009 images belonging to 10 classes.


In [14]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/5
18009/18009 [==============================] - 308s - loss: 0.2003 - acc: 0.9467 - val_loss: 1.4922 - val_acc: 0.5873
Epoch 2/5
18009/18009 [==============================] - 301s - loss: 0.1771 - acc: 0.9550 - val_loss: 1.3471 - val_acc: 0.6292
Epoch 3/5
18009/18009 [==============================] - 299s - loss: 0.1500 - acc: 0.9621 - val_loss: 1.3366 - val_acc: 0.6122
Epoch 4/5
18009/18009 [==============================] - 296s - loss: 0.1383 - acc: 0.9635 - val_loss: 1.2506 - val_acc: 0.6496
Epoch 5/5
18009/18009 [==============================] - 297s - loss: 0.1318 - acc: 0.9649 - val_loss: 1.1890 - val_acc: 0.6462
